# Churn case. Part 1.

In [1]:
import pandas as pd

## Reading all CSV files with Pandas

In [2]:
customer = pd.read_csv("customer.csv")
customer.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes


In [3]:
cust_loc = pd.read_csv("cust_loc.csv")
cust_loc.head()

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [4]:
cust_services = pd.read_csv("cust_services.csv")
cust_services.head()

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [5]:
cust_account = pd.read_csv("cust_account.csv")
cust_account.head()

,Account_id,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges
0,3668-QPYBK,2,Month-to-month,Mailed check,Yes,53.85,108.15
1,9237-HQITU,2,Month-to-month,Electronic check,Yes,70.70,151.65
2,9305-CDSKC,8,Month-to-month,Electronic check,Yes,99.65,820.5
3,7892-POOKP,28,Month-to-month,Electronic check,Yes,104.80,3046.05
4,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3


In [6]:
cust_account.dtypes

Account_id           object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges         object
dtype: object

In [7]:
cust_churn = pd.read_csv("cust_churn.csv")
cust_churn.head()

,Id,Churn
0,3668-QPYBK,Yes
1,9237-HQITU,Yes
2,9305-CDSKC,Yes
3,7892-POOKP,Yes
4,0280-XJGEX,Yes


## Create database and tables

In [8]:
import sqlite3

In [9]:
conn = sqlite3.connect("churn.db")
cur = conn.cursor()

In [11]:
# Create customer table
customer.to_sql("customer", conn, if_exists="replace", index=False)

In [12]:
# Create customer location table
cust_loc.to_sql("cust_loc", conn, if_exists="replace", index=False)

In [13]:
# Create customer services table
cust_services.to_sql("cust_services", conn, if_exists="replace", index=False)

In [14]:
# Create customer accountcust_account table
cust_account.to_sql("cust_account", conn, if_exists="replace", index=False)

In [15]:
# Create customer churn table
cust_churn.to_sql("cust_churn", conn, if_exists="replace", index=False)

## Information about our tables

In [10]:
def table_info(conn,cursor):
    """
    prints out all of the columns of every table in the DB
    
    conn: database connection object
    cursor: cursor object
    """
    
    tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print("\t "+ col)
            print()

In [11]:
table_info(conn,cur)

customer
	 CustomerID

	 Gender

	 SeniorCitizen

	 Partner

	 Dependents

cust_loc
	 Cust_ID

	 State

	 Latitude

	 Longitude

	 ZipCode

cust_services
	 Cust_ID

	 PhoneService

	 MultipleLines

	 InternetService

	 OnlineSecurity

	 OnlineBackup

	 DeviceProtection

	 TechSupport

	 StreamingTV

	 StreamingMovies

cust_account
	 Account_id

	 Tenure

	 Contract

	 PaymentMethod

	 PaperlessBilling

	 MonthlyCharges

	 TotalCharges

cust_churn
	 Id

	 Churn



## Join all tables

In [12]:
cur.execute(
"""
ALTER TABLE cust_loc
RENAME COLUMN Cust_ID TO CustomerID;
""")

In [13]:
table_info(conn,cur)

customer
	 CustomerID

	 Gender

	 SeniorCitizen

	 Partner

	 Dependents

cust_loc
	 CustomerID

	 State

	 Latitude

	 Longitude

	 ZipCode

cust_services
	 Cust_ID

	 PhoneService

	 MultipleLines

	 InternetService

	 OnlineSecurity

	 OnlineBackup

	 DeviceProtection

	 TechSupport

	 StreamingTV

	 StreamingMovies

cust_account
	 Account_id

	 Tenure

	 Contract

	 PaymentMethod

	 PaperlessBilling

	 MonthlyCharges

	 TotalCharges

cust_churn
	 Id

	 Churn



In [14]:
cur.execute(
"""
SELECT 
*
FROM customer
INNER JOIN cust_loc
USING (CustomerID)
""")
cur.fetchone()

('3668-QPYBK',
 'Male',
 'No',
 'No',
 'No',
 'California',
 33.964131,
 -118.272783,
 90003)

In [15]:
cust_loc = cust_loc.rename(columns={"Cust_ID": "CustomerID"})

In [16]:
cust_loc.head()

,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [17]:
pd.concat([customer,cust_loc], join="inner", axis=1)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,Male,No,No,No,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,Female,No,No,Yes,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,Female,No,No,Yes,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,Female,No,Yes,Yes,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,Male,No,No,Yes,0280-XJGEX,California,34.039224,-118.266293,90015
...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Female,No,No,No,2569-WGERO,California,34.341737,-116.539416,92285
7039,6840-RESVB,Male,No,Yes,Yes,6840-RESVB,California,34.667815,-117.536183,92301
7040,2234-XADUH,Female,No,Yes,Yes,2234-XADUH,California,34.559882,-115.637164,92304
7041,4801-JZAZL,Female,No,Yes,Yes,4801-JZAZL,California,34.167800,-116.864330,92305


In [18]:
# Rename remaining columns from tables
cust_services = cust_services.rename(columns={"Cust_ID": "CustomerID"})
cust_account = cust_account.rename(columns={"Account_id": "CustomerID"})
cust_churn = cust_churn.rename(columns={"Id": "CustomerID"})

In [19]:
dfs_to_join = [customer, cust_loc, cust_services, cust_account, cust_churn]

In [20]:
customer.loc[customer["Gender"] == "Male",]

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
4,0280-XJGEX,Male,No,No,Yes
6,8779-QRDMV,Male,Yes,No,No
7,1066-JKSGK,Male,No,No,No
8,6467-CHFZW,Male,No,Yes,Yes
...,...,...,...,...,...
7034,3605-JISKB,Male,Yes,Yes,No
7035,9767-FFLEM,Male,No,No,No
7036,8456-QDAVC,Male,No,No,No
7039,6840-RESVB,Male,No,Yes,Yes


In [21]:
churn_all = pd.concat(dfs_to_join, join="inner", axis=1)

In [22]:
churn_all = churn_all.loc[:, ~churn_all.columns.duplicated()]

In [23]:
churn_all.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.5,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3,Yes


In [24]:
churn_all.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [25]:
churn_all["TotalCharges"] = pd.to_numeric(churn_all["TotalCharges"], errors="coerce")

In [26]:
churn_all.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [27]:
# Missing values
churn_all.isnull().sum(axis=0)

CustomerID           0
Gender               0
SeniorCitizen        0
Partner              0
Dependents           0
State                0
Latitude             0
Longitude            0
ZipCode              0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Tenure               0
Contract             0
PaymentMethod        0
PaperlessBilling     0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [28]:
# Drop the missing values
churn_all = churn_all.dropna()

In [29]:
churn_all.isnull().sum(axis=0)

CustomerID          0
Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
State               0
Latitude            0
Longitude           0
ZipCode             0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Tenure              0
Contract            0
PaymentMethod       0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [30]:
churn_all.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [31]:
churn_all.to_sql("churn_all", conn, if_exists="replace", index=False)

7032

In [32]:
cur.execute("PRAGMA table_info('churn_all')").fetchall()

[(0, 'CustomerID', 'TEXT', 0, None, 0),
 (1, 'Gender', 'TEXT', 0, None, 0),
 (2, 'SeniorCitizen', 'TEXT', 0, None, 0),
 (3, 'Partner', 'TEXT', 0, None, 0),
 (4, 'Dependents', 'TEXT', 0, None, 0),
 (5, 'State', 'TEXT', 0, None, 0),
 (6, 'Latitude', 'REAL', 0, None, 0),
 (7, 'Longitude', 'REAL', 0, None, 0),
 (8, 'ZipCode', 'INTEGER', 0, None, 0),
 (9, 'PhoneService', 'TEXT', 0, None, 0),
 (10, 'MultipleLines', 'TEXT', 0, None, 0),
 (11, 'InternetService', 'TEXT', 0, None, 0),
 (12, 'OnlineSecurity', 'TEXT', 0, None, 0),
 (13, 'OnlineBackup', 'TEXT', 0, None, 0),
 (14, 'DeviceProtection', 'TEXT', 0, None, 0),
 (15, 'TechSupport', 'TEXT', 0, None, 0),
 (16, 'StreamingTV', 'TEXT', 0, None, 0),
 (17, 'StreamingMovies', 'TEXT', 0, None, 0),
 (18, 'Tenure', 'INTEGER', 0, None, 0),
 (19, 'Contract', 'TEXT', 0, None, 0),
 (20, 'PaymentMethod', 'TEXT', 0, None, 0),
 (21, 'PaperlessBilling', 'TEXT', 0, None, 0),
 (22, 'MonthlyCharges', 'REAL', 0, None, 0),
 (23, 'TotalCharges', 'REAL', 0, None, 0

In [33]:
pd.read_sql("select * from churn_all", conn)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.30,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,2569-WGERO,Female,No,No,No,California,34.341737,-116.539416,92285,Yes,...,No internet service,No internet service,No internet service,72,Two year,Bank transfer (automatic),Yes,21.15,1419.40,No
7028,6840-RESVB,Male,No,Yes,Yes,California,34.667815,-117.536183,92301,Yes,...,Yes,Yes,Yes,24,One year,Mailed check,Yes,84.80,1990.50,No
7029,2234-XADUH,Female,No,Yes,Yes,California,34.559882,-115.637164,92304,Yes,...,No,Yes,Yes,72,One year,Credit card (automatic),Yes,103.20,7362.90,No
7030,4801-JZAZL,Female,No,Yes,Yes,California,34.167800,-116.864330,92305,No,...,No,No,No,11,Month-to-month,Electronic check,Yes,29.60,346.45,No


## See all tables

In [34]:
cur.execute("select name from sqlite_master where type='table'")
cur.fetchall()

[('customer',),
 ('cust_loc',),
 ('cust_services',),
 ('cust_account',),
 ('cust_churn',),
 ('churn_all',)]